# Predicting back pain with `sklearn` and `GridSearchCV`
I'm going to look at some medical biometric data, and then predict whether an individual is going to sugger from abnormal back pain that was seen in the patients previously using these measurements. It is quite a small data set, so I will be doing some preprocessing and model building and comparison `sklearn`; I'm hoping that this will act as a good reference point for people using `sklearn`. 

I'm going to focus on these main algorithms for binary classification problems:
1. Decision trees
2. Logisitic regression 
3. Support Vector Machines 

If you are interested in seeing a kernel written in `R` on this dataset that uses `caret` then I have written one [here](https://www.kaggle.com/willcanniford/predicting-lower-back-pain-using-caret-in-r). I wrote that `R` one first, and during my Python learning, I have expanded that and decided to produce this kernel as well to transfer the findings and models into a different programming language.   

Using `GridSearchCV` here I implement cross-validation search across parameters for certain `sklearn` models and then investigate the information provided by this feedback to try and show the advantage of exploring different metrics when choosing final model parameters. 

In [ ]:
# Import packages
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import numpy as np 
import pandas as pd 

## Reading in and cleaning our data 
Firstly, we are going to read in the data and clean it so that we are able to work with it with more clarity, and to make the model building stage easier. You'll see that the last column of the data that is given to us contains the names of the other columns in the set. We can use this information to name the columns, and then drop this 'meta' column from the data frame. 

In [ ]:
raw_data = pd.read_csv('../input/Dataset_spine.csv')

# View the column names given in the last column
raw_data.iloc[:, -1].head(20)

While we are cleaning the data, and renaming the columns using the last column's information, we will also split the data into feature variables and response. This is going to make it easier for us later when we are actually modelling the data. With `sklearn` it requires us to have separated the labels from the data and pass them in separately.  

In [ ]:
# Remove the last column that only contains meta data for us
data = raw_data.iloc[:, :-1]

# View the data prior to renaming the columns for comparison
print(data.head())

In [ ]:
# Rename the columns in place using a dictionary and the information found in the 13th column
meta_column_names = {"Col1" : "pelvic_incidence", "Col2" : "pelvic_tilt",
                     "Col3" : "lumbar_lordosis_angle","Col4" : "sacral_slope", 
                     "Col5" : "pelvic_radius","Col6" : "degree_spondylolisthesis", 
                     "Col7" : "pelvic_slope","Col8" : "direct_tilt", 
                     "Col9" : "thoracic_slope", "Col10" :"cervical_tilt", 
                     "Col11" : "sacrum_angle", "Col12" : "scoliosis_slope"}

# Rename the columns using the above dictionary, and replace the existing data object with inplace = True
data.rename(columns = meta_column_names, inplace = True)

Let's split the data into features (`X`) and response (`y`). Note that we have already removed the 'meta' column, so we don't have to worry about that, and can just take the final column of the dataset that represents whether the patient suffered from abnormal back pain or not (`Class_att`).  

Let's also just check that the dimensions of the two variables match up. We need the features to have the same number of observations/rows as the labels that are contained with `y`. 

In [ ]:
# Features
X = data.iloc[:, :-1]
# Response
y = data.iloc[:, -1]

print(X.shape)
print(y.shape)

That all looks good (we know that we have 310 observations, and 12 features); we can have a final look at the data as a whole, but just keep in mind that we are going to mainly working with the `X` and `y` variables that we have defined. 

In [ ]:
data.head()

# Visualising the data 
One of the traditional starting points of any analysis, is the visualisation of the data to try and get a feel for the values, and the interactions between some of the features. 

## Visualising the correlations between the feature variables with `.corr()`
Now that we have cleaned up the column names, we can have a quick look at the diferent feature variables present in the data and their interactions, as well as try and visualise our information prior to building and comparing our different prediction models. A useful correlation visualisation is using `.corr()` on the dataset. If you have a high number of feature variables, then it is probably for the best to filter the columns prior to using the visualisation, otherwise it might become too cluttered to be informative. 

In [ ]:
correlations = X.corr()
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(correlations, square=True, cbar=True, annot=True, vmax=.9);

## Further visualising correlations with `seaborn`
We can expand on our previous plot using the `sns.pairplot()` function from `seaborn`. We can see that the first 6 columns seem to hold higher correlations between them, while the remaining variables are seemingly unrelated, Let's have a look at the pairplot of those first 6 columns. 

In [ ]:
sns.pairplot(X.iloc[:, :6], height = 1.75)
plt.show()

## Visualising differences between groups
While it is important to note the correlations between variables, our main goal is to predict whether an individual observation is likely to suffer from normal or abnormal pain. This means that is probably more informative to look through the feature variables with them split by group so that we find particular feature variables that are of immediate interest to us regarding predictive power. 

There are further techniques that involve the combination of feature variables to improve the predictive power of a model; this could either be multiplicative between certain variables or additive. I may explore this at a later date to see if we can further improve the models' performance. 

In [ ]:
fg, ax = plt.subplots(ncols=4, nrows=3, sharex=False, sharey=False, figsize=(15,12))
fg.tight_layout()
# Add h spacing and space at the bottom for the legend
plt.subplots_adjust(hspace=0.25, bottom = 0.1)

# If we reshape the 2D numpy array that is returned from subplots for the axes then we can loop through without worrying about dimensions
for index, axes in enumerate(ax.reshape(-1)):
    normal = sns.distplot(data[data.Class_att == 'Normal'].iloc[:, index], ax=axes, color='#2980b9', label = 'Normal')
    abnormal = sns.distplot(data[data.Class_att == 'Abnormal'].iloc[:, index], ax=axes, color='#e74c3c', label = 'Abnormal')
    
# For this to work you have to have named the lines AND given labels 
# Get the handles and lines from the final axes - we can do this in this case because the lines are the same for every subplot
h, l = axes.get_legend_handles_labels()
fg.legend(h, l, loc='lower center', ncol = 2)
plt.show()

The largest disparity that we see for a single variable between 'Normal' and 'Abnormal' is 'degree_spondylolisthesis'; let's visualise that on its own using a swarm plot. 

In [ ]:
sns.swarmplot(y='degree_spondylolisthesis', x='Class_att', data=data)
plt.title('degree_spondylolisthesis difference between classes\n')
plt.xlabel('')
plt.ylabel('')
plt.show()

While this highlights the difference in levels between the two classes, it also makes apparent that we have more "Abnormal" cases than "Normal" in this dataset. 

# Training our models
Let's start having a look at predicting the values that we are interested in. 

## Scaling the data using `StandardScaler` 
I'm going to scale the feature variables below using the `StandardScaler` from `sklearn.preprocessing`. This won't necessarily change the result of effectiveness of every algorithm, but those that are distance based will be regularised so that the scale of the variable doesn't impact it's respective weight in the classification stage. i.e. those variables that are generally larger will hold more weight when the classification is made if the data isn't scaled. We want all the variables on the same scale. 

If we were later to recieve more data, as you would imagine a model like this might be used in a real-world application, then you can use the same `StandardScaler` object to scale that incoming data to be used with our model. This would likely be wrapped into a single package or cleaning function that you could reuse as the data was fed into the system. 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)

In [ ]:
type(X_scaled)

You'll note here that a `numpy.ndarray` has been returned, rather than a data frame. We have also lost our column names. If you want to have the scaled data in a `pd.DataFrame` then you'll have to rebuild it using the column names pulled from the original `X` data frame. I've written an example below of how you might do this if you wanted to easily visualise etc. 

In [ ]:
scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
scaled_df['Class_att'] = y
scaled_df.head()

## Train test split
Even though this dataset is small, I think that it is still best practice to implement a training and test split of the data to try and get a better indication of out-of-sample prediction after the model has been built.

Now that we can see that we have scaled the features, and encoded the response, we can create our training and test splits of the data using `sklearn.model_selection.train_test_split`.  We are going to be performing a simple 75% train, 25% test split with our data. I have used a `random_state` to ensure that the resuls are consistent throughout the kernel, and that every time it is run the results are going to be the same for everyone that views this. 

I am also going to quickly demonstrate the difference between using `stratify` during the class to `train_test_split` to make sure that the classes are even between both the training and test sets. It is a small point here, but may be more important with other data sets to avoid the training set being unbalanced, or the test set not containing any rare positive cases for example. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25, random_state = 1000)

In [ ]:
proportion_train = round(len(y_train[y_train == 'Abnormal'])/len(y_train)*100, 2)
proportion_test = round(len(y_test[y_test == 'Abnormal'])/len(y_test)*100, 2)
print('Train case proportion: {}%'.format(proportion_train))
print('Test case proportion:  {}%'.format(proportion_test))
print('Difference between test and train sets: {}%'.format(round(np.abs(proportion_train - proportion_test), 2)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25, random_state = 1000, stratify = y)

In [ ]:
proportion_train = round(len(y_train[y_train == 'Abnormal'])/len(y_train)*100, 2)
proportion_test = round(len(y_test[y_test == 'Abnormal'])/len(y_test)*100, 2)
print('Train case proportion: {}%'.format(proportion_train))
print('Test case proportion:  {}%'.format(proportion_test))
print('Difference between test and train sets: {}%'.format(round(np.abs(proportion_train - proportion_test), 2)))

We can see that our classes are more evenly balanced between the train and test set when we specify the response variable in the `stratify` argument. Now that we have our training and testing sets all finalised, we can get on with training and evaluating our models. 

## Decision Tree model using `DecisionTreeClassifier`
An incredibly popular model for binomial prediction problems in the decision tree, so it seems like a good place to start. This is just a singular tree with default hyper parameters so isn't necessary reflective of the ideal model or performance that could be achieved using this particular algorithm. 

In [ ]:
# Import decision trees and accuracy metric 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

We have imported our `DecisionTreeClassifier`, and now can fit the model using training data, and measure the model's effectiveness using the test data. 

In [ ]:
# We are going to be doing this a lot, so better to define a function
def print_accuracy(accuracy_score, score_text=False):
    """
    Take an accuracy score between 0 and 1 and print output to screen cleanly 
    """
    clean_accuracy = accuracy_score*100.0
    if score_text:
        clean_text = score_text.strip() + ' '
        print('{}{:.2f}%'.format(clean_text, clean_accuracy))
    else:
        print('{:.2f}%'.format(clean_accuracy))

In [ ]:
tree_model = DecisionTreeClassifier(random_state = 1000).fit(X_train, y_train)

# Predict on the test set and get the accuracy using the known values 
tree_pred = tree_model.predict(X_test)

print_accuracy(accuracy_score(y_test, tree_pred), 'Decision tree accuracy:')

Note that not specifying any hyperparameters with `DecisionTreeClassifier` basically means it will keep growing until all the nodes are pure, which will likely result in huge overfitting issues. We should do some hyperparameter training to see if there is a more optimum combination of hyperparameters that will produce a better model for accuracy on our test set. 

### Tuning our `DecisionTreeClassifier` using `GridSearchCV` 
In addition to our test-train split we are going to perform cross validation. This takes a number of folds from the data and trains a model on one section before testing on the other, on the training subset. The final model that is returned is a model that uses the best hyperparameters for that full testing set and is trained on the full set. This internal test from within the cross validation set is sometimes referred as the validation set; it is the performance on this set that will give us insight into the performance of the model on unseen examples outside of the training examples.  

We can then test this further because we will have already separated out a final test set for us to use; this might lead to some interesting results because the data size is so small, but I think that it is good practice so will do it regardless. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define our parameter grid that will be looped through to test our hyperparameters
param_grid = {'max_depth': [i for i in range(1, 11)], 
              'max_features': [i for i in range(1, 8)], 
              'min_samples_leaf': [i for i in range(1, 11)]}

grid = GridSearchCV(DecisionTreeClassifier(random_state = 1000), param_grid, cv=10, return_train_score = True)

Once the grid object has been created, we can fit it using our train data to create a model and find the best hyperparameters according to the out-of-sample tests done during the cross-validation. 

In [ ]:
grid.fit(X_train, y_train)

By accessing our `grid` object we can find out the best out-of-sample score that we obtained, as well as the parameters that provided that; the `best_score_` is defined as _mean cross-validated score of the best_estimator_, so we can see what the average score across all cross-validations for our 'optimum' parameters is. This might act as a good indication of the model's future performance on unseen data. Further to that, we can access the best estimator model through the `best_estimator_` portion of the object. I'll use that to predict our test set and see what accuracy score we get.  

In [ ]:
print("Best Score: {}%".format(round(grid.best_score_*100.0, 2)))
print("Best params: {}".format(grid.best_params_))

In [ ]:
print_accuracy(accuracy_score(y_test, grid.best_estimator_.predict(X_test)), 'Decision Tree Classifier:')

The accuracy that we saw on the test set wasn't as good as what we might have expected given the cross-validation score that we had; but it is still good, and not a bad starting point! 

## Investigating the performance using `cv_results_`
If we wanted to know more about the performance of the model during the process, then we could take a look for ourselves into the scores that were generated during this process. Perhaps we didn't want the particular set of hyperparameters that had the best mean score, but instead wanted a set of hyperparameters that was very consistent in its predictive power so we might better be able to predict performance in our testing set. Since the training times are recorded as well, we are able to observe the tradeoff between certain parameter settings and the performance of the model. More information about this can be found in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

It should also be noted that we can define a scorer of our choosing, and it might be more appropriate to use a balanced accuracy measure here to avoid scores being higher purely due to picking the majority class each time; this is something that I may implement in the next version of the kernel. 

If you have set `return_train_score = True` then the results of the cross-validation will be stored as below:

In [ ]:
keys = []
shapes = []
examples = []
for key in list(grid.cv_results_.keys()):
    keys.append(key)
    shapes.append(len(grid.cv_results_.get(key)))
    examples.append(grid.cv_results_.get(key)[0])

df_results_info = pd.DataFrame({'size': shapes, 'example': examples}, index = keys)
df_results_info

In [ ]:
plt.hist(grid.cv_results_.get('mean_fit_time'), bins = 70);plt.show()

In [ ]:
mean_fit_times = grid.cv_results_.get('mean_fit_time')
mean_test_scores = grid.cv_results_.get('mean_test_score')
max_score = np.argmax(mean_test_scores)

plt.scatter(mean_fit_times, mean_test_scores)
plt.xlim(min(mean_fit_times)*0.95, max(mean_fit_times*1.05))
plt.scatter(mean_fit_times[max_score], mean_test_scores[max_score])
plt.show()

Above I have highlighted the singular point that represents the highest mean test score. To continue what I was saying, it might be the case that you want to investigate the performance of a model computationally, which might be a consideration when you are training on larger models, or have a flow of data that requires retraining periodically. 

In [ ]:
max_depth = grid.cv_results_.get('param_max_depth').astype(int)
max_features = grid.cv_results_.get('param_max_features').astype(int)
min_samples_leaf = grid.cv_results_.get('param_min_samples_leaf').astype(int)
mean_fit_times = grid.cv_results_.get('mean_fit_time')
mean_test_scores = grid.cv_results_.get('mean_test_score')

params_df = pd.DataFrame({'max_depth' : max_depth, 
                          'max_features' : max_features, 
                          'min_samples_leaf' : min_samples_leaf,
                          'mean_fit_time' : mean_fit_times,
                          'mean_test_score' : mean_test_scores})

In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols = 3, figsize = (15,5), sharey=True, sharex=True)
plt.xlim(min(mean_fit_times)*0.95, max(mean_fit_times*1.05))

features = ['max_depth', 'max_features', 'min_samples_leaf']
axes = ax.flatten()

for idx in range(3):
    sns.scatterplot(y = 'mean_test_score', x = 'mean_fit_time', hue = features[idx], data = params_df, ax = axes[idx], palette='Greens_r')
    
plt.show()

__Note:__ if the above graphs look to have any major outliers, then this might be due to the particular run of the grid when this notebook was rendered. 

Looking these graphs, sorted and coloured by the 3 parameters that were part of our grid, we can see that `max_features` seems to have a strong impact on the fitting time of a model, and that by limiting that to lower values provides using with a sizeable computational performance improvement, whereas the other two have a more spread set across the `mean_fit_time` in comparison; they still have some patterns that we can see.  

`min_samples_leaf` tends to take longer the lower it is which makes sense given that the lower this parameter is, then the deeper the final model will be. `max_depth` also appears to show that the higher the depth, then the longer the model takes to train. 

## Logistic Regression model with `LogisticRegression`
Now we have built a model using a decision tree, we can build another model using logistic regression for comparison. 

In [ ]:
from sklearn.linear_model import LogisticRegression

 By altering the `penalty` attribute of the `LogisiticRegression` we can change the type of regularisation that the model uses. This can have an impact on the effectiveness and efficiency of the model, and could be included as one of the hyperparameters that we tune when we run the cross-validation and `gridSearch`.

In [ ]:
param_grid = {'penalty': ['l1', 'l2']}
grid = GridSearchCV(LogisticRegression(random_state = 1000, solver = 'liblinear'), param_grid, cv=10, return_train_score=True)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print("Best Score: {}".format(grid.best_score_))
print("Best params: {}".format(grid.best_params_))

In [ ]:
print_accuracy(accuracy_score(y_test, grid.best_estimator_.predict(X_test)), 'Best logisitic regression grid score:')

Just for the sake of comparison and because we are only using a small parameter grid (two levels), I will create a model using the `l1` regularisation penalty to show how it differs from `l2`. Here lies an example of the benefit of stating the `random_state` argument when creating the `LogisticRegression` model. 

In [ ]:
# Solver suppresses the warnings: more information about which solver for which problems can be found in the docs: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
log_reg_l1 = LogisticRegression(penalty='l1', solver = 'liblinear', random_state=1000)
log_model_l1 = log_reg_l1.fit(X_train, y_train)
y_pred_l1 = log_model_l1.predict(X_test)
print(log_model_l1.coef_)

print_accuracy(accuracy_score(y_test, y_pred_l1), 'L1 Regularised test accuracy:')

In [ ]:
# Having a look at the l2 regularisation, which isn't capable of fully removing features
log_reg_l2 = LogisticRegression(penalty='l2', solver = 'liblinear', random_state=1000)
log_model_l2 = log_reg_l2.fit(X_train, y_train)
y_pred_l2 = log_model_l2.predict(X_test)
print(log_model_l2.coef_)

print_accuracy(accuracy_score(y_test, y_pred_l2), 'L2 Regularised test accuracy:')

Note how L1 regularisation of the model acts as feature selection as it has reduced one coefficient of our features to 0 and thus removed them from the model. This is a benefit to using the L1 regularisation as in this way it is capable of feature selection and maintaining the simplicity of the model. 

In [ ]:
grid.best_estimator_.coef_

In [ ]:
log_model_l2.coef_

In [ ]:
log_model_l2.coef_ == grid.best_estimator_.coef_

This is just to show to that the model returned by the grid serach is the same as that returned by the model that we built externally afterwards.  

Let's take a look at the results that were captured during the grid search results and see how the models performed at every split. 

In [ ]:
test_splits = [i for i in list(grid.cv_results_.keys()) if i.endswith('test_score') and i.startswith('split')]
test_split_results = {}
penalty = list(grid.cv_results_.get('param_penalty'))

for split in test_splits:
    test_split_results[split] = list(grid.cv_results_.get(split))
    
df_test_splits = pd.DataFrame(test_split_results, index = penalty)
df_test_splits.columns = [i.replace('_test_score', '') for i in df_test_splits.columns]

In [ ]:
df_test_splits

In [ ]:
df_test_splits.loc['l1'] >= df_test_splits.loc['l2']

We can see that `L1` regularisation actually performed better or equal in 80% of the splits that we performed, and that the average performance was only marginally less overall for the test sets. 

In [ ]:
list(grid.cv_results_.get('mean_test_score'))

In [ ]:
long_test_splits = pd.DataFrame(df_test_splits.unstack().reset_index())
long_test_splits.columns = ['split', 'penalty', 'score']
long_test_splits

In [ ]:
sns.boxplot(x='penalty', y="score", data=long_test_splits)
plt.xlabel('Penalty')
plt.ylabel('Test score')
plt.show()

We can see a tighter clustering of the `L2` test scores, while the `L1` results appear to be generally higher but the average brought down by two outliers; with more cross-validation passes this might have shown that `L1` was the more effective model. 

## Support Vector Machine model with `SVC`
When considering using support vector machines, we are going to look at the performance of two different types: Polynomial and RBF. You can see how these are defined below, as well as their performance against the test set that we have defined.  

Support vector machines can be a good method of creating interesting relationships by introducing polynomial features that allow for the creating a line that separates the two classes. RBF (radial basis function) methods are more popular, and are density based. The gamma hyperparameter is important in the battle of fighting overfitting. A higher value of gamma will make a tighter fit around the data points, but this means that it won't necessarily generalise well, whereas accuracy is likely to fall with a lower gamma value, but it might mean that performance generalises better to unseen observations.  

In [ ]:
# Import support vector machines
from sklearn.svm import SVC

In [ ]:
poly_model = SVC(kernel = 'poly', gamma = 'auto', random_state = 1000).fit(X_train, y_train)
rbf_model = SVC(kernel = 'rbf', gamma = 'auto', random_state = 1000).fit(X_train, y_train)

poly_pred = poly_model.predict(X_test)
print_accuracy(accuracy_score(y_test, poly_pred), 'Polynomial kernel accuracy:')

rbf_pred = rbf_model.predict(X_test)
print_accuracy(accuracy_score(y_test, rbf_pred), 'RBF kernel accuracy:')

With the default settings we can see that the RBF performed much better, let's have a look and see whether tweaking the value of gamma will make a noticeable difference to the performance on the test set. 'auto' for gamma uses a value of 1/n_features by [default](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), so lets produce a range that is around that to see if there are any improvements to be made.   

When we are using a grid search technique then we are selecting the best hyperparameters based on their performance on generalising outside of the data on which they have been trained, so a lower gamma makes sense as we want to avoid overfitting the data to the training set. 

In [ ]:
param_grid = {'gamma': np.arange(0, 0.25, 0.01)}
grid = GridSearchCV(SVC(random_state = 1000, kernel = 'rbf'), param_grid, cv=5, return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
print("Best Score: {}".format(grid.best_score_))
print("Best params: {}".format(grid.best_params_))
print(1/12)

By default the gamma would have been 1/12, or ~0.08. Using the `GridSearch` we are able to see that a lower gamma value of 0.06 might perform better on unseen data; we can see how it performs on the test that we have by manually specifying that gamma value of 0.06 and rebuilding the model. 

In [ ]:
rbf_model_2 = SVC(kernel = 'rbf', gamma = 0.03, random_state = 1000).fit(X_train, y_train)
rbf_pred_2 = rbf_model_2.predict(X_test)

print_accuracy(accuracy_score(y_test, rbf_pred), 'Original RBF kernel accuracy:')
print_accuracy(accuracy_score(y_test, rbf_pred_2), 'RBF kernel accuracy with gamma of 0.06:')

Our `GridSearch` has yielded good results, and changing that gamma value has meant that the performance is slightly better on our test set, going from **76.92%** to **78.21%**. 

In [ ]:
gammas = list(grid.cv_results_.get('param_gamma'))
mean_test_scores = list(grid.cv_results_.get('mean_test_score'))
mean_train_times = list(grid.cv_results_.get('mean_fit_time'))

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 1, sharex = True, sharey = False)
axs = axes.flatten()

plt.sca(axs[0])
sns.lineplot(gammas, mean_test_scores, color = '#44bd32')
plt.axvline(0.06, color = '#fbc531')
plt.ylabel('Mean Test Score')
plt.title('Mean Test Score during CV as Gamma increases')

plt.sca(axs[1])
sns.lineplot(x=gammas, y=mean_train_times, color="#8c7ae6")
plt.ylabel('Mean Training Time')
plt.axvline(0.06, color = '#fbc531')
plt.xlabel('Gamma value')
plt.show()

It looks as though we have a good value for gamma in terms of both the test score that was achieved via the cross-validation, and the training time for the model. 

# Conclusions
We can see that early into this investigation that both logistic regression models are performing better than our decision tree and our support vector machines; at least with the tuning that I have done. I think this is a good example of where throwing complexity at a problem doesn't always yield results. Sometimes this is important when you have large amounts of data, and have issues training computationally; this isn't a problem with our small dataset but could prove something to consider when the data size becomes larger so looking at the training information that can be provided through the `cv_results_` of a `GridSearchCV` object is useful, and I hope that I've shown some of the ways in which that might shape the final decision of choosing hyperparameters.  

__Coming soon__
- Other algorithms to explore for hyperparameter tuning - Ensemble methods
- Binary problem accuracy methods

If you've found this useful then please consider leaving an upvote or a comment. Please let me know if you see anything that I could improve or if you have constructive feedback. 


If you are interested in seeing a similar kernel written in`R` on this dataset that uses `caret` then I have written one with just that [here](https://www.kaggle.com/willcanniford/predicting-lower-back-pain-using-caret-in-r).
